In [0]:
from pyspark.sql import functions as F
from delta.tables import DeltaTable

df_chamado = spark.read.table("v_credit.silver.tb_chamado")

TABELA_DESTINO = "v_credit.gold.dm_chamado"

In [0]:
df_gold = df_chamado.select(
    F.col("cd_chamado").cast("bigint").alias("cd_chamado"),

    F.coalesce(F.col("tm_espera"), F.lit(0)).cast("bigint").alias("tm_espera"),
    F.coalesce(F.col("tm_duracao"), F.lit(0)).cast("bigint").alias("tm_duracao")
)


DeltaTable.forName(spark, TABELA_DESTINO).alias("t").merge(
    df_gold.alias("s"), "t.cd_chamado = s.cd_chamado"
).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

print(f"✅ Dimensão Chamado carregada com sucesso!")